<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-файла-и-изучение-данных" data-toc-modified-id="Загрузка-файла-и-изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка файла и изучение данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков,-редактирование-ошибочных-значений" data-toc-modified-id="Обработка-пропусков,-редактирование-ошибочных-значений-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков, редактирование ошибочных значений</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Анализ-платежеспособности-клиентов" data-toc-modified-id="Анализ-платежеспособности-клиентов-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ платежеспособности клиентов</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Исследование надёжности заёмщиков

Датасет представляет собой данные о клиентах банка и статистику по их платежеспособности.

**Цель**: сделать предобработку данных и разобраться, влияют ли различные параметры клиента (семейное положение, количество детей) на факт погашения кредита в срок, есть ли зависимость между доходом клиента и фактом погашения кредита в срок, как разные цели кредита влияют на факт погашения кредита в срок.

# Описание данных

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Загрузка файла и изучение данных

In [1]:
# импортируем нужные библиотеки
import pandas as pd
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem
from collections import Counter
import numpy as np

# открываем файл с датасетом, создаем копию оригинального датасета для сравнения 
# с исправленным и предварительно проверяем и исследуем его

try:
    credit = pd.read_csv('C:/Jupyter_projects/datasets/1_data.csv')
    credit_original = pd.read_csv('C:/Jupyter_projects/datasets/1_data.csv')
except:
    credit = pd.read_csv('/datasets/data.csv')
    credit_original = pd.read_csv('/datasets/data.csv')

credit.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# выводим размерность таблицы
print(f'Размерность датасета: {credit.shape}')
print('')
credit.info()

Размерность датасета: (21525, 12)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# выводим статистические параметры датасета
credit.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Выводы**

1. Данные загружены: количество и названия столбцов совпадают с теми, что были даны в теоретическом описании проекта, данных по количеству строк не было предоставлено;
2. Предварительно были замечены следующие проблемы с данными, требующие решения:
    - отрицательные значения в столбце **days_employed** и **children**;
    - нереальные аномально высокие *положительные* значения в столбце **days_employed**: 340266 дней стажа = 340266/247 (число рабочих дней в году) = 1626 года стажа (при возрасте 53 года). При этом отрицательные значения по модулю выглядят адекватно;
    - строки, где стаж **days_employed**, пересчитанный в года, больше возраста **dob_years**;
    - нули в столбце **dob_years** можно расценивать как *пропущенные значения*;
    - текст в разном регистре в столбцах **education** и **family_status**;
    - вероятно синонимичные словосочетания в столбце **purpose**, которые нужно привести к одной категории (например, "сыграть свадьбу" и "на проведение свадьбы", "покупка жилья" и "покупка жилья для семьи");
    - подозрительное значение в столбце **gender**;
    - подозрительные значения в столбце **children**: при возрасте = 21 children = 20;
3. Данные содержат 21525 строк и 12 столбцов;
4. Данные содержат форматы трех типов: int64 (целочисленный 64бит - 5 столбцов), float64 (вещественный 64бит - 2 столбца) и object (строковый - 5 столбцов);
5. Помимо столбца **dob_years**, где нулевые значения можно расценивать как пропущенные, есть два столбца данных, содержащих пропущенные значения в виде NaN: **days_employed** и **total_income**. Итого **3 столбца с пропусками**;
6. Названия столбцов записаны подходящим образом;

## Предобработка данных

### Обработка пропусков, редактирование ошибочных значений

- Целью данной главы является обработка пропущенных значений, а также редактирование ошибочных значений, которые сложно представить в реальной жизни;

- Начнем с того, что для более удобной работы заменим нулевые значения в столбце с возрастом на объект nan;

In [4]:
# нули в столбце dob_years заменяем на np.nan
credit['dob_years'] = credit['dob_years'].replace(0, np.nan)

In [5]:
# считаем количество NaN в столбцах данных
credit.isna().sum()

children               0
days_employed       2174
dob_years            101
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

- У нас два столбца с одинковым количеством строк с пропущенных значений: по **2174 пропусков**. Интересно посмотреть, совпадают ли эти строки для двух столбцов;
- Но сначала посмотрим, совпадают ли строки с пропусками в столбце возраста (**101 пропуск**) со строками остальных столбцов с пропусками;

In [6]:
# выведем NaN
credit[credit['dob_years'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,NaN,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,NaN,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,NaN,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,NaN,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,NaN,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,-934.654854,NaN,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,370879.508002,NaN,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,-5043.219890,NaN,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,NaN,NaN,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
1898,0,370144.537021,NaN,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


In [7]:
# создадим отдельные df для строк с NaN из столбца days_employed и total_income
credit_na_days_employed = credit[credit['days_employed'].isna()]
credit_na_total_income = credit[credit['total_income'].isna()]

# проверка на идентичность датафреймов
display(credit_na_days_employed.equals(credit_na_total_income)) # проверяем равенство
credit[(credit['days_employed'] == np.nan) 
       & (credit['total_income'] != np.nan)
      ].empty #дополнительная проверка: 
# пуст ли df с условием, что в одном столбце NaN, а в другом - нет

True

True

- Таким образом, 101 пропуск в столбце возраста не имеет очевидной связи с пропусками других столбцов;
- Строки с пропущенными значениями в столбцах стажа и дохода совпадают. Вероятно есть какая-то взаимосвязь между этими столбцами, но она скорее связана с получением (выгрузкой) или обработкой исходных полученных данных, т.к. нет очевидной логической связи между этими двумя столбцами;

In [8]:
# выведем сегмент датафрейма с NaN
credit_na_total_income.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65.0,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41.0,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63.0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50.0,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54.0,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21.0,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52.0,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32.0,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50.0,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52.0,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


- Пропуски данных видятся **полностью случайными**, т.к. нет очевидной зависимости вероятности встретить данные пропуски от других данных. Вероятно, среди столбцов стажа и дохода один из них зависит от другого (доход пересчитывается из стажа?), но с имеющейся информацией сложно делать такие выводы;
- Пропущенными данными во всех столбцах - **количественные**;
- Строки с пропущенными данными в столбце возраста составляют < 1% от общего объема данных, заменим эти значения на среднее значение возраста;  
- Строки с пропущенными данными в столбцах стажа и дохода представляют собой ~ 10% данных, поэтому просто убирать их нецелесообразно по причине потери довольно большой части данных;
- Данные по стажу и доходу вряд ли получится получить из других источников, предположить их, следуя опыту или знаниям области не представляется возможным;
- Возможным вариантом может быть построение линейной регрессии *стаж* vs *доход* по имеющимся данным и заполнить пропуски, следуя полученному уравнению регрессии. Вероятно, нужно построить такую регрессию для разных типов занятости для более точного учета дохода и стажа, что займет довольно много времени, при этом нет гарантий, что мы получим хорошую корреляцию;
- Учитывая вшесказанное и ограниченное количество времени, **оптимальная стратегия** для обработки пропущенных значений в нашем случае - **замена на характерное значение выборки**: среднее или медиану. К тому же пропуски вероятно полностью случайные, поэтому выбранная стратегия не должна координально поменять основные статистические параметры выборки;

- Заменяем пропуски столбца возраста на среднее значение имеющихся данных;
- Посмотрим диапазоны значений стажа и дохода для определения более подходящего параметра для замены пропущенных значений: среднее или медиана;
- Сначала нужно избавиться от отрицательных чисел в столбце стажа. Вероятно, минус был случайно добавлен в часть данных, потому что по модулю отрицательные значения выглядят адекватно;
- Также нужно разобраться с тем фактом, что по модулю именно отрицательные числа выглядят адекватно, в то время как положительные выглядят неправдоподобно. Нереальные завышенные положительные значения к тому же будут сильно влиять на среднее, если мы выберем среднее в качестве замены пропущенным значениям;

In [9]:
# считаем среднее значение dob_years
mean_age = credit['dob_years'].mean()

# заполняем NaN столбца dob_years на среднее значение
credit['dob_years'].fillna(mean_age, inplace=True)

# проверяем результаты
credit.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [10]:
# считаем количество положительных значений в столбце days_employed и их долю во всем датасете
print('Общее количество проблемных значений: {:.2%}'.format(credit[credit['days_employed'] > 0]['days_employed'].count()/len(credit)))

Общее количество проблемных значений: 16.00%


In [11]:
# сгруппируем по income_type тех, у кого days_employed > 0, и посчитаем их количество
print('Для положительных значений:')
display(credit[credit.days_employed > 0].groupby('income_type')['income_type'].count())

# посчитаем общее количество пенсионеров и безработных по столбцу days_employed
print('Для всех значений:')
print('безработный: ', credit[credit.income_type == 'безработный']['days_employed'].count())
print('пенсионер: ', credit[credit.income_type == 'пенсионер']['days_employed'].count())

Для положительных значений:


income_type
безработный       2
пенсионер      3443
Name: income_type, dtype: int64

Для всех значений:
безработный:  2
пенсионер:  3443


- Общее количество проблемных завышенных значений столбца со стажем от объема всего датасета ~ 16%, что довольно много для простого удаления строк с данными значениями;
- Положительные завышенные значения столбца со стажем встречаются для *пенсионеров* и *безработных* по типу занятости;
- Похоже, что положительные завышенные значения стажа - это стаж не в днях, а в **часах**: при делении положительных значений  на 24 получается диапазон от 13 000 до 16 000 дней стажа, что хорошо согласуется с максимумом данных с адекватным значению по модулю - 18 000 дней, если учесть, что практически все строки интервала 13 000 - 16 000 имеют тип занятости пенсионер, то данные значения выглядят вполне приемлемыми;
- Данные по пенсионерам и безработным скорее всего заполняются немного иным алгоритмом, нежели данные по остальным: для пенсионеров и безработных ошибочно считаются **часы стажа**, а не дни, а для остальных стаж в днях ошибочно **умножается на -1**;
- Таким образом, решение проблемных значений (пока не включая пропуски) столбца со стажем: **положительные значения** делим на 24 (переводим из часов в дни), берем модуль **отрицательных значений**;

In [12]:
# пишем функцию для редактирования столбца days_employed
def days_employed_editing(days):
    """
    редактирует столбец days_employed
    
    params: 
    - days - значение столбца days_employed для каждой строки
    
    returns:
    - при days > 0 возвращает результат деления days на 24 (переводим из часов в дни) 
    - при days < 0 возвращает модуль days
    - при days = 0 возвращает days
    """
    if days > 0:
        return days/24
    elif days < 0:
        return abs(days)
    else:
        return days

# редактируем столбец days_employed и проверим результаты
credit['days_employed'] = credit['days_employed'].apply(days_employed_editing)
credit['days_employed'].describe()

count    19351.000000
mean      4641.641176
std       5355.964289
min         24.141633
25%        927.009265
50%       2194.220567
75%       5537.882441
max      18388.949901
Name: days_employed, dtype: float64

In [13]:
# выведем диапазоны значений для столбцов с NaN: days_employed и total_income
credit[['days_employed', 'total_income']].describe()

,days_employed,total_income
count,19351.000000,1.935100e+04
mean,4641.641176,1.674223e+05
std,5355.964289,1.029716e+05
min,24.141633,2.066726e+04
25%,927.009265,1.030532e+05
50%,2194.220567,1.450179e+05
75%,5537.882441,2.034351e+05
max,18388.949901,2.265604e+06


- Проблемные значения в столбцах со стажем обработаны;
- Теперь можно вернуться к заполнению пропусков в столбцах со стажем и доходом;
- Исходя из разброса значений (3 порядка для стажа и 2 порядка для дохода), целесообразнее использовать медиану в качестве замены пропусков;
- В обоих столбцах пропущено примерно по 10% данных. Это слишком большая доля, чтобы адекватно заполнить ее единственным характерным значением — мы наверняка исказим распределение. В таком случае лучше взять несколько характерных значений по группам - значениям столбца типа занятости;

In [14]:
# извлекаем значения столбца income_type, для которых total_income = NaN
total_income_nan_groups = credit[credit['total_income'].isna()]['income_type'].value_counts().index
total_icnome_original_median = credit['total_income'].median()

# итерируем по значениям total_income_nan_groups:
for i in total_income_nan_groups:
    # считаем медиану по total_income для каждой группы столбца income_type
    group_median = credit[credit['income_type'] == i]['total_income'].median()
    # выбираем строки с NaN в столбце total_income с определенным значением income_type и заменяем NaN на соответствующую медиану
    credit.loc[(credit['total_income'].isna()) & 
               (credit['income_type'] == i), 
               'total_income'] = group_median

# проверим результаты
display(credit.isna().sum())
print('Медиана значений столбца total_income ДО: {:.2f}'.format(total_icnome_original_median))
print('Медиана значений столбца total_income ПОСЛЕ: {:.2f}'.format(credit['total_income'].median()))

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Медиана значений столбца total_income ДО: 145017.94
Медиана значений столбца total_income ПОСЛЕ: 142594.40


In [15]:
# извлекаем значения столбца income_type, для которых days_employed = NaN
days_employed_nan_groups = credit[credit['days_employed'].isna()]['income_type'].value_counts().index

# итерируем по значениям days_employed_nan_groups:
for i in days_employed_nan_groups:
    # считаем медиану по days_employed для каждой группы столбца income_type
    group_median = credit[credit['income_type'] == i]['days_employed'].median()
    # выбираем строки с NaN в столбце days_employed с определенным значением income_type и заменяем NaN на соответствующую медиану
    credit.loc[(credit['days_employed'].isna()) & 
               (credit['income_type'] == i), 
               'days_employed'] = group_median

# проверим результаты
display(credit.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

- Замена пропущенных значений завершена;
- Были встречены строки, где стаж превышает возраст, исследуем данную проблему, присвоив строкам категорию в зависимости от того, превышает ли стаж максимальный стаж. Установим максимальный стаж не 16 лет, а 14 лет, т.к. большая часть людей - пенсионеры, и, если мы рассматриваем Россию, то пенсионеры начинали свою трудовую деятельность еще в СССР, где разрешалось работать с 14 лет;
- Для пересчета дней стажа в года используем 365 дней;

In [16]:
# пересчитываем дни в года и записываем значения в новый столбец years_employed
credit['years_employed'] = credit['days_employed']/365

# пишем функцию для присвоения строке категории стажа
def experience_flag(row):
    """
    присваивает категорию, в зависимости от соотношения стажа и максимального стажа
    
    params:
    - row - строка датафрейма
    
    returns:
    - строку 'нереальный', если стаж больше максимального стажа
    - строку 'реальный', если стаж меньши или равен максимальному стажу
    """
    experience = row['years_employed']
    age = row['dob_years']
    max_experience = age - 14
    if experience > max_experience:
        return 'нереальный'
    else:
        return 'реальный'

# расставим категории стажа и выведем количество строк в категории нереальный
credit['experience_flag'] = credit.apply(experience_flag, axis=1)
display(credit['experience_flag'].value_counts())

# посчитаем процент строк из категории нереальный от общего числа
print('Доля строк с категорией стажа "нереальный": {:.2%}'.format(credit[credit['experience_flag'] == 'нереальный']['experience_flag'].count()/credit.shape[0]))

реальный      20514
нереальный     1011
Name: experience_flag, dtype: int64

Доля строк с категорией стажа "нереальный": 4.70%


- Сложно придумать, как можно исправить строки, где стаж привышает максимальный стаж. С учетом того, что таких строк всего 4%, принимается решение об их удалении из датасета;

In [17]:
# убираем строки с категорией стажа нереальный
print('Строк с категорией стажа "нереальный" ДО: ', credit[credit['experience_flag'] == 'нереальный']['experience_flag'].count())
credit.drop(credit[credit['experience_flag'] == 'нереальный'].index, inplace=True)
print('Строк с категорией стажа "нереальный" ПОСЛЕ: ', credit[credit['experience_flag'] == 'нереальный']['experience_flag'].count())
print(f'Количество строк датасета: {credit.shape[0]}')

Строк с категорией стажа "нереальный" ДО:  1011
Строк с категорией стажа "нереальный" ПОСЛЕ:  0
Количество строк датасета: 20514


- С данными по нереальному стажу разобрались;
- Далее обработаем отрицательные значения в столбце количество детей (children);

In [18]:
# заменим отрицательные значения в столбце children и проверим результат
credit['children'] = credit['children'].apply(abs)
credit['children'].describe()

count    20514.000000
mean         0.559520
std          1.387484
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

- Существует также данные, где у клиентов есть по 20 детей;
- Для женщины родить 20 детей - физиологически архисложная задача, но у нас нет информации, влючены ли сюда приемные дети, поэтому теоретически такая ситуация может быть;
- Другой момент: у нас данные по количеству детей идут по порядку от 0 до 5 и затем сразу 20, без промежуточных значений, что довольно сложно представить в рамках такого случайного события, как обращения людей за кредитом;
- Строк, где количество детей равно 20, не так уж и много, поэтому можно просто выкинуть их все из датасета;

In [19]:
# выведем долю строк с количеством детей = 20
print('Доля строк количество детей равным 20: {:.2%}'.format(credit[credit['children'] == 20]['children'].count()/credit.shape[0]))

Доля строк количество детей равным 20: 0.36%


In [20]:
# удаляем строки, где children = 20
print('Количество проблемных значений: ', credit[credit['children'] == 20]['children'].count())
print('Строк, включая проблемными значениями: ', credit['children'].count())
credit.drop(credit[credit['children'] == 20].index, inplace=True)
print('Строк без проблемных значений: ', credit['children'].count())

Количество проблемных значений:  73
Строк, включая проблемными значениями:  20514
Строк без проблемных значений:  20441


In [21]:
# выводим значения столбца gender
display(credit.gender.value_counts())

# удаляем проблемные значения столбца gender
print(credit.shape)
credit.drop(credit[credit['gender'] == 'XNA'].index, inplace=True)
print(credit.shape)

F      13364
M       7076
XNA        1
Name: gender, dtype: int64

(20441, 14)
(20440, 14)


**Выводы**

1. Проблемные значения исправлены в столбцах **days_employed**, **children** и **income_type**;
2. Пропущенные значения найдены и обработаны в столбцах **days_employed**, **dob_years** и **total_income**;
3. Во всех столбцах пропуски являются **количественными**;
4. Пропуски в столбце **dob_years** были обработаны удалением строк с этими пропусками, т.к. доля таких строк составляет < 1% от всех данных;
5. Пропуски в столбцах **days_employed** и **total_income** были обработаны заменой медианой имеющехся данных столбцов, т.к. доля строк с пропусками в этих столбцах составляет ~ 10% от всех данных, что много для обычного удаления, медиана выбрана по причине довольно большого разброса значений данных столбцов (3 и 2 порядка соответственно);

### Замена типа данных

- Все данные вещественного типа (дни, года, доход) обычно представлены в целочисленном виде, поэтому целью данной главы является преобразование числовых данных из вещественного в целочисленный тип;

In [22]:
# переведем значения столбцов days_employed, total_income, dob_years и years_employed из float64 в int64
credit['days_employed'] = credit['days_employed'].astype('int')
credit['total_income'] = credit['total_income'].astype('int')
credit['dob_years'] = credit['dob_years'].astype('int')
credit['с'] = credit['years_employed'].astype('int')

# проверяем результаты
credit.info()
credit.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20440 entries, 0 to 21524
Data columns (total 15 columns):
children            20440 non-null int64
days_employed       20440 non-null int64
dob_years           20440 non-null int64
education           20440 non-null object
education_id        20440 non-null int64
family_status       20440 non-null object
family_status_id    20440 non-null int64
gender              20440 non-null object
income_type         20440 non-null object
debt                20440 non-null int64
total_income        20440 non-null int64
purpose             20440 non-null object
years_employed      20440 non-null float64
experience_flag     20440 non-null object
с                   20440 non-null int64
dtypes: float64(1), int64(8), object(6)
memory usage: 2.5+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,experience_flag,с
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23.116912,реальный,23
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11.026860,реальный,11
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15.406637,реальный,15
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11.300677,реальный,11
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38.843159,реальный,38


**Выводы**

1. Изначально столбцы **days_employed** и **total_income** имели вещественный тип данных, после замены нулевых значений на np.nan в столбце **dob_years**, данные этого столбца сменились с целочисленного на вещественный;
2. Данные по дням стажа, доходу и возрасту обычно хранятся целыми числами;
3. Чтобы преобразовать тип данных этих столбцов из вещественного в целочисленный, был использован метод **astype()**, т.к. он, в отличие от метода *to_numeric()*, может приводить данные к целочисленному типу;

### Обработка дубликатов

- Целью данной главы является поиск и удаление дубликатов в данных;

In [23]:
# посчитаем количество полных дубликатов
print('Количество полных дубликатов ДО: ', credit.duplicated().sum())

# убираем все полные дубликаты, обновляя индексы и убирая столбец со старыми индексами
credit = credit.drop_duplicates().reset_index(drop=True)

print('Количество полных дубликатов ПОСЛЕ: ', credit.duplicated().sum())

Количество полных дубликатов ДО:  53
Количество полных дубликатов ПОСЛЕ:  0


In [24]:
# приведем к нижнему регистру значения столбцов education и family_status
credit.education = credit.education.str.lower()
credit.family_status = credit.family_status.str.lower()

# проверим результаты
display(credit.education.value_counts())
credit.family_status.value_counts()

среднее                14319
высшее                  5065
неоконченное высшее      729
начальное                268
ученая степень             6
Name: education, dtype: int64

женат / замужем          11716
гражданский брак          3960
не женат / не замужем     2706
в разводе                 1137
вдовец / вдова             868
Name: family_status, dtype: int64

In [25]:
# посчитаем полные дубликаты
print('Количество полных дубликатов ДО: ', credit.duplicated().sum())

# убираем оставшиеся полные дубликаты, обновляя индексы и убирая столбец со старыми индексами
credit = credit.drop_duplicates().reset_index(drop=True)

# посчитаем количество полных дубликатов
print('Количество полных дубликатов ПОСЛЕ: ', credit.duplicated().sum())

Количество полных дубликатов ДО:  17
Количество полных дубликатов ПОСЛЕ:  0


**Выводы**

1. Изначально были найдены 54 полных дубликата (**ручной поиск дубликатов**), однако, после приведению к нижнему регистру значений столбцов, где регистр был разный, было обнаружено еще 17 полных дубликатов (**ручной поиск дубликатов с учетом регистра**) - итого 71 полный дубликат были удалены из данных;
2. Возможные причины появления дубликатов в данных: 
    - случайное копирование строки;
    - обработка заявки клиента двумя разными операторами; 
    - отправка заявки несколько раз клиентом (если онлайн через приложение/сайт);
    - перебои с интернетом клиента или проблемы с сервером банка, в результате чего запрос отправился более одного раза;

### Лемматизация

- Целью лемматизации для данного датасета является выделение категорий из целей кредита, т.к. среди указанных целей кредита довольно много синонимичных формулировок, которые затрудняют анализ данных;

In [26]:
# создаем объект лемматизации
lemmatizer = Mystem()

# сохраняем список лемм в отдельный столбец и выводим пять строк
credit['lemmas_purpose'] = credit['purpose'].apply(lemmatizer.lemmatize)
credit.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,experience_flag,с,lemmas_purpose
6811,1,984,33,среднее,1,гражданский брак,1,F,компаньон,1,156076,свадьба,2.696693,реальный,2,"[свадьба, \n]"
324,0,1447,30,высшее,0,в разводе,3,F,сотрудник,1,353864,операции с коммерческой недвижимостью,3.966480,реальный,3,"[операция, , с, , коммерческий, , недвижимо..."
10101,0,2847,48,среднее,1,женат / замужем,0,F,сотрудник,0,238303,строительство жилой недвижимости,7.802340,реальный,7,"[строительство, , жилой, , недвижимость, \n]"
2974,0,8123,51,среднее,1,женат / замужем,0,F,компаньон,0,263243,на покупку своего автомобиля,22.255358,реальный,22,"[на, , покупка, , свой, , автомобиль, \n]"
18343,0,14579,60,высшее,0,вдовец / вдова,2,F,пенсионер,0,114308,недвижимость,39.944033,реальный,39,"[недвижимость, \n]"


In [27]:
# получение списка всех лемм как отдельных элементов списка
lemmas_single = []
for i in credit.lemmas_purpose:
    for k in i:
        lemmas_single.append(k)

In [28]:
# создаем список уникальных лемм с их частотой, убирая лишнее (пробелы, переходы строки, предлоги и т.д.)
lemmas_unique = list(Counter(lemmas_single).most_common())
lu = lemmas_unique[:]
for i in lu:
    if len(i[0]) < 5:
        lemmas_unique.remove(i)

# выведем список уникальных лемм
display(lemmas_unique)

# выбираем основные уникальные леммы и помещаем их в список
lemmas_most_common = ['недвижимость', 'автомобиль', 'образование', 'свадьба', 'ремонт', 'жилье']
lemmas_most_common

[('недвижимость', 6025),
 ('покупка', 5602),
 ('жилье', 4217),
 ('автомобиль', 4110),
 ('образование', 3817),
 ('операция', 2465),
 ('свадьба', 2201),
 ('строительство', 1774),
 ('высокий', 1299),
 ('коммерческий', 1248),
 ('получение', 1240),
 ('жилой', 1173),
 ('сделка', 904),
 ('дополнительный', 863),
 ('заниматься', 863),
 ('сыграть', 726),
 ('проведение', 722),
 ('сдача', 609),
 ('семья', 604),
 ('собственный', 594),
 ('ремонт', 579),
 ('подержанный', 462),
 ('подержать', 452),
 ('приобретение', 443),
 ('профильный', 419)]

['недвижимость', 'автомобиль', 'образование', 'свадьба', 'ремонт', 'жилье']

In [29]:
# пишем функцию для определения категории из списка уникальных лемм по цели получения кредита
def purpose_flag(lemmas_list):
    """
    присвивает категорию по цели получения кредита для строки датафрейма на основе содержания в строке той или иной 
    уникальной леммы
    
    params:
    - lemmas_list - список лемм цели получения кредита строки
    
    returns:
    - i - выделенная уникальная лемма
    """
    for i in lemmas_most_common:
        if i in lemmas_list:
            return i

# определяем категории purpose_flag
credit['purpose_flag'] = credit['lemmas_purpose'].apply(purpose_flag)

# заменим категорию жилье на недвижимость
credit.loc[credit['purpose_flag'] == 'жилье', 'purpose_flag'] = 'недвижимость'

# проверим результаты
credit['purpose_flag'].value_counts()

недвижимость    9663
автомобиль      4110
образование     3817
свадьба         2201
ремонт           579
Name: purpose_flag, dtype: int64

**Выводы**

1. Лемматизация выполнена для **выделения категорий по целям получения кредита**;
2. Следующие шаги были выполнены в процессе лемматизации:
    - выделение лемм для каждой строки;
    - добавление списка лемм каждой строки в отдельный столбец;
    - выделение наиболее уникальных лемм и сортировка их по частоте втречаемости;
    - дополнительный анализ списка уникальных лемм и выделение минимального количества, способного описать все цели получения кредита;
    - присвоение категорий согласно тому, какая уникальная лемма встречена в цели получения кредита для конкретной строки;
3. Итого выделены следующие категории целей получения кредита: *недвижимость*, *автомобиль*, *образование*, *свадьба*, *ремонт*;

### Категоризация данных

- Категоризация данных уже была частично проведена на шагах *2.1 Обработка пропусков, редактирование ошибочных значений* и *2.4 Лемматизация*;
- Для редактрования ошибочных значений датасет был помечен двумя категориями, связанных со стажем: *реальный* стаж, где указанный стаж был меньше или равен максимально возможному стажу, и *нереальный* стаж, где указанный стаж был больше максимально возможного стажа;
- Лемматизация была использована, чтобы перейти от большого количества синонимичных категорий исходного столбца с целью получения кредита к более емким и всеобъемлющим категориям на основе лемм столбца с целями получения кредита;

- Для более удобного анализа столбца с категориями целей получения кредита **purpose_flag** создадим столбец с id для каждой категории для более удобного доступа, а потом создадим словарь для этого столбца, а также для других столбцов с категориальными данными: **education** и **family_status**. Кроме того, один из вопросов затрагивает зависимость дохода от того, будет ли возвращен кредит в срок, для этого будет удобнее разбить доход на категории;

In [30]:
# выводим категории в столбце purpose_flag
credit.purpose_flag.unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба', 'ремонт'],
      dtype=object)

In [31]:
# создадим список кортежей с id категории и ее название
purpose_flag_ids = [(pid, name) for pid, name in enumerate(list(credit.purpose_flag.unique()))]
purpose_flag_ids

[(0, 'недвижимость'),
 (1, 'автомобиль'),
 (2, 'образование'),
 (3, 'свадьба'),
 (4, 'ремонт')]

In [32]:
# напишим функцию для присвоения id соответствующей категории в строке датафрейма
def purpose_flag_id(flag):
    """
    присваивает id в зависимости от категории в столбце purpose_flag
    
    params:
    - flag - строковое значение категории из строки датафрейма
    
    returns:
    - i[0] - id категории
    """
    for i in purpose_flag_ids:
        if flag in i[1]:
            return i[0]
        
# определим id для категорий из столбца purpose_flag и проверим результаты
credit['purpose_flag_id'] = credit['purpose_flag'].apply(purpose_flag_id)
display(credit.purpose_flag.value_counts())
display(credit.purpose_flag_id.value_counts())
credit.head(10)

недвижимость    9663
автомобиль      4110
образование     3817
свадьба         2201
ремонт           579
Name: purpose_flag, dtype: int64

0    9663
1    4110
2    3817
3    2201
4     579
Name: purpose_flag_id, dtype: int64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,experience_flag,с,lemmas_purpose,purpose_flag,purpose_flag_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23.116912,реальный,23,"[покупка, , жилье, \n]",недвижимость,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11.026860,реальный,11,"[приобретение, , автомобиль, \n]",автомобиль,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15.406637,реальный,15,"[покупка, , жилье, \n]",недвижимость,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11.300677,реальный,11,"[дополнительный, , образование, \n]",образование,2
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38.843159,реальный,38,"[сыграть, , свадьба, \n]",свадьба,3
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,2.537495,реальный,2,"[покупка, , жилье, \n]",недвижимость,0
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,7.888225,реальный,7,"[операция, , с, , жилье, \n]",недвижимость,0
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0.418574,реальный,0,"[образование, \n]",образование,2
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,18.985932,реальный,18,"[на, , проведение, , свадьба, \n]",свадьба,3
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5.996593,реальный,5,"[покупка, , жилье, , для, , семья, \n]",недвижимость,0


In [33]:
# создадтим предобработанную копию датасета
credit_processed = credit.copy(deep=True)

# для удобства поменяем порядок столбцов и сохраним этот порядок в датафрейме credit
credit_columns = ['children', 'days_employed', 'years_employed', 'dob_years', 'experience_flag', 
                  'education', 'education_id', 'family_status', 'family_status_id', 'gender', 
                  'income_type', 'debt', 'total_income', 'purpose', 'purpose_flag', 'purpose_flag_id', 
                  'lemmas_purpose']
credit = credit[credit_columns]

# проверим порядок
credit.head(10)

,children,days_employed,years_employed,dob_years,experience_flag,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_flag,purpose_flag_id,lemmas_purpose
0,1,8437,23.116912,42,реальный,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,0,"[покупка, , жилье, \n]"
1,1,4024,11.026860,36,реальный,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1,"[приобретение, , автомобиль, \n]"
2,0,5623,15.406637,33,реальный,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,0,"[покупка, , жилье, \n]"
3,3,4124,11.300677,32,реальный,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,2,"[дополнительный, , образование, \n]"
4,0,14177,38.843159,53,реальный,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,3,"[сыграть, , свадьба, \n]"
5,0,926,2.537495,27,реальный,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,0,"[покупка, , жилье, \n]"
6,0,2879,7.888225,43,реальный,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,0,"[операция, , с, , жилье, \n]"
7,0,152,0.418574,50,реальный,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,2,"[образование, \n]"
8,2,6929,18.985932,35,реальный,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,3,"[на, , проведение, , свадьба, \n]"
9,0,2188,5.996593,41,реальный,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,0,"[покупка, , жилье, , для, , семья, \n]"


In [34]:
# выведем статистику по столбцу total_income
credit['total_income'].describe()

count    2.037000e+04
mean     1.665203e+05
std      9.898426e+04
min      2.066700e+04
25%      1.083892e+05
50%      1.428125e+05
75%      1.967368e+05
max      2.265604e+06
Name: total_income, dtype: float64

- Чтобы данные равномерно попали в наши категории, подберем значения для разделения дохода на три категории на основе квантилей : возьмем в качестве граничных значений 0.33 и 0.66;

In [35]:
# извлекаем квантили столбца total_income
total_income_quantiles = list(credit['total_income'].quantile([0.33, 0.66]))
q_lower = total_income_quantiles[0]
q_upper = total_income_quantiles[1]

# проверим результаты
total_income_quantiles

[120361.34000000001, 172357.0]

In [36]:
# напишем функцию для присвоения категории по значению столбца total_income
def total_income_flag(income):
    """
    присваивает категорию по значению в столбце total_income
    
    params:
    - income - значение total_income строки
    
    returns:
    - соответствующую категорию
    """
    if income < q_lower:
        return 'низкий'
    elif income > q_lower and income < q_upper:
        return 'средний'
    else:
        return 'высокий'

# проверим результат
credit['total_income_flag'] = credit['total_income'].apply(total_income_flag)
display(credit['total_income_flag'].value_counts())
credit.head()

высокий    7239
низкий     6722
средний    6409
Name: total_income_flag, dtype: int64

,children,days_employed,years_employed,dob_years,experience_flag,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_flag,purpose_flag_id,lemmas_purpose,total_income_flag
0,1,8437,23.116912,42,реальный,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,0,"[покупка, , жилье, \n]",высокий
1,1,4024,11.026860,36,реальный,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1,"[приобретение, , автомобиль, \n]",низкий
2,0,5623,15.406637,33,реальный,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,0,"[покупка, , жилье, \n]",средний
3,3,4124,11.300677,32,реальный,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,2,"[дополнительный, , образование, \n]",высокий
4,0,14177,38.843159,53,реальный,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,3,"[сыграть, , свадьба, \n]",средний


In [37]:
# создаем словари для purpose_flag
purpose_flag_dict = credit[['purpose_flag_id', 'purpose_flag']].drop_duplicates().reset_index(drop=True)
education_dict = credit[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
family_status_dict = credit[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)

display(purpose_flag_dict)
display(education_dict)
family_status_dict

,purpose_flag_id,purpose_flag
0,0,недвижимость
1,1,автомобиль
2,2,образование
3,3,свадьба
4,4,ремонт


,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,3,в разводе
3,4,не женат / не замужем
4,2,вдовец / вдова


**Выводы**

1. Категории целей получения кредита получили свои уникальные id и были помечены в данных;
2. Созданы словари для категориальных данных следующих столбцов: **purpose_flag**, **education** и **family_status**
3. Классификация выполнена по типу: выделены уникальные категории с присвоенными им id;

## Анализ платежеспособности клиентов

*Есть ли зависимость между наличием детей и возвратом кредита в срок?*

- Т.к. размер каждой группы разный, то для сравнения количество должников каждой группы мы должны нормировать данные на размер группы, т.е. сравнивать доли должников каждой отдельной группы;

In [46]:
# посчитаем долю должников по children
display(credit.pivot_table(index='children', 
                   values = 'debt', 
                   aggfunc= lambda x: x.sum()/x.count()
                  ).sort_values(by='debt', ascending=False))

# сгруппируем по столбцу children и посчитаем общее количество клиентов для каждой группы
credit.groupby('children')['children'].count()

,debt
children,
4,0.100000
2,0.094164
1,0.092327
3,0.080000
0,0.076743
5,0.000000


children
0    13213
1     4744
2     2039
3      325
4       40
5        9
Name: children, dtype: int64

**Вывод**

- Видно, что **доля должников возрастает с увеличением количества детей в семье**. Похоже, многодетные семьи реже возвращают кредит в срок. Возможно это связано с социальными особенностями в России, где чаще многодетные семьи малообеспеченные финансово, т.к. чем больше детей, тем больше расходов, поэтому и отдвать кредит становится сложнее;
- Из данного тренда выбивается лишь доля должников с 5 детьми, она равна 0. Посмотрев на данные, мы можем заметить, строк с со значением количества детей равным 5 всего лишь 9, ни один из этих клиентов не имеет задолжности. Это довольно маленькая выборка, поэтому полагаться на значения, связанные со строками, где количество детей равно 5, не стоит;

*Есть ли зависимость между семейным положением и возвратом кредита в срок?*

In [40]:
# посчитаем долю должников по family_status
credit.pivot_table(index='family_status', 
                   values = 'debt', 
                   aggfunc= lambda x: x.sum()/x.count()
                  ).sort_values(by='debt', ascending=False)

# сгруппируем по столбцу family_status и посчитаем средний возраст
credit.groupby('family_status')['dob_years'].mean().sort_values()

family_status
не женат / не замужем    0.099039
гражданский брак         0.093465
женат / замужем          0.077022
в разводе                0.071240
вдовец / вдова           0.062212
Name: debt, dtype: float64

family_status
не женат / не замужем    38.196600
гражданский брак         41.862969
женат / замужем          43.334472
в разводе                45.639402
вдовец / вдова           57.074885
Name: dob_years, dtype: float64

**Вывод**

- Люди, **не вступавшие в брак**, и люди **в гражданском браке реже возвращают кредит в срок**;
- Возможной причиной такой картины является тот факт, что люди не вступавшие в брак и в гражданском браке в среднем моложе остальных, молодые - менее мудрые и опытные и более рискованные, отсюда и больше плохо спланированных кредитов;

*Есть ли зависимость между уровнем дохода и возвратом кредита в срок?*

In [43]:
# посчитаем долю должников по total_income_flag
credit.pivot_table(index='total_income_flag', 
                   values = 'debt', 
                   aggfunc= lambda x: x.sum()/x.count()
                  ).sort_values(by='debt', ascending=False)

,debt
total_income_flag,
средний,0.089562
низкий,0.083309
высокий,0.074596


**Вывод**

- C небольшим отрывов люди со средним доходом обычно чаще не возвращают кредит в срок, но в целом **ситуация** более менее **равная**, поэтому можно сделать вывод, что **уровень дохода не сильно влияет** на то, будет ли **кредит возвращен в срок**;

*Как разные цели кредита влияют на его возврат в срок?*

In [45]:
# посчитаем долю должников по purpose_flag
credit.pivot_table(index='purpose_flag', 
                   values = 'debt', 
                   aggfunc= lambda x: x.sum()/x.count()
                  ).sort_values(by='debt', ascending=False)

,debt
purpose_flag,
образование,0.094053
автомобиль,0.093917
свадьба,0.078146
недвижимость,0.074821
ремонт,0.058722


**Вывод**

- Похоже, что кредиты **на образование и автомобиль реже всего возращаются в срок**;

## Общий вывод

- По результатам анализа скорее всего **не вернут кредит в срок** люди не состоящие в браке либо состоящие в гражданском браке, из многодетных семей и берущие кредит на образование или автомобиль;
- Уровень дохода при это особо не влияет на способность вернуть кредит в срок;